In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import scipy.stats
from prettytable import PrettyTable
%matplotlib inline

In [2]:
pip install imblearn

     |████████████████████████████████| 167 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 7.2 MB 804 kB/s eta 0:00:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2
You should consider upgrading via the '/usr/local/opt/python/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
dg = pd.read_csv('LRM_1_hour.csv')

In [8]:
dg.columns

Index(['Unnamed: 0', 'uhid', 'pulserate', 'ecg_resprate', 'spo2', 'heartrate',
       'mean_bp', 'sys_bp', 'dia_bp', 'peep', 'pip', 'map', 'tidalvol',
       'minvol', 'ti', 'fio2', 'abd_difference_y', 'abdomen_girth',
       'currentdateheight', 'currentdateweight', 'dischargestatus', 'new_ph',
       'rbs', 'stool_day_total', 'temp', 'total_intake',
       'totalparenteralvolume', 'tpn-tfl', 'typevalue_Antibiotics',
       'typevalue_Inotropes', 'urine', 'gender', 'birthweight', 'birthlength',
       'birthheadcircumference', 'inout_patient_status', 'gestationweekbylmp',
       'gestationdaysbylmp', 'baby_type', 'central_temp', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'gestation'],
      dtype='object')

In [10]:
def logistic(X_train, X_test, y_train):
    model = LogisticRegression()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test) 
    return y_pred

In [15]:
cols = ['pulserate_SE',
       'pulserate_DFA', 'pulserate_ADF', 'pulserate_Mean', 'pulserate_Var',
       'ecg_resprate_SE', 'ecg_resprate_DFA', 'ecg_resprate_ADF',
       'ecg_resprate_Mean', 'ecg_resprate_Var', 'spo2_SE', 'spo2_DFA',
       'spo2_ADF', 'spo2_Mean', 'spo2_Var', 'heartrate_SE', 'heartrate_DFA',
       'heartrate_ADF', 'heartrate_Mean', 'heartrate_Var', 'peep_SE',
       'peep_DFA', 'peep_ADF', 'peep_Mean', 'peep_Var', 'pip_SE', 'pip_DFA',
       'pip_ADF', 'pip_Mean', 'pip_Var', 'map_SE', 'map_DFA', 'map_ADF',
       'map_Mean', 'map_Var', 'tidalvol_SE', 'tidalvol_DFA', 'tidalvol_ADF',
       'tidalvol_Mean', 'tidalvol_Var', 'minvol_SE', 'minvol_DFA',
       'minvol_ADF', 'minvol_Mean', 'minvol_Var', 'ti_SE', 'ti_DFA', 'ti_ADF',
       'ti_Mean', 'ti_Var', 'fio2_SE', 'fio2_DFA', 'fio2_ADF', 'fio2_Mean',
       'fio2_Var', 'abdomen_girth', 'urine', 'total_intake',
       'totalparenteralvolume', 'new_ph', 'gender', 'birthweight',
       'birthlength', 'birthheadcircumference', 'inout_patient_status',
        'baby_type', 'apgar_onemin',
       'apgar_fivemin', 'apgar_tenmin', 'motherage', 'conception_type',
       'mode_of_delivery', 'steroidname', 'numberofdose', 'rbs', 'temp',
       'currentdateweight', 'currentdateheight',  'gestation',
        'tpn-tfl', 'abd_difference', 'stool_day_total',
       'typevalue_Antibiotics', 'typevalue_Inotropes','mean_bp', 'sys_bp',
       'dia_bp']

In [16]:
def training(X,y):
    auc_roc = []
    training = []

    for i in range(25):
        #try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.3 
                                                        )
        # SMOTE
        sm = SMOTE()
        X_train_sm, y_train_sm = sm.fit_sample(X_train, y_train)
        y_smote = logistic(X_train_sm, X_test, y_train_sm)
        y_smote_1 = logistic(X_train_sm,X_train,y_train_sm)
        auc_roc.append(roc_auc_score(y_test,y_smote))
        training.append(roc_auc_score(y_train,y_smote_1))
        continue
    return auc_roc,y_test,y_smote
        #except:
        #    continue

In [17]:
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return round(m,3), round(m-h,3), round(m+h,3)

In [18]:
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_a = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_1 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 6

In [28]:
dg = pd.read_csv('LRM_6_hour.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_6 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_6 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_6 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [31]:
dg = pd.read_csv('LRM_12_hour_2.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_12 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_12 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_12 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [32]:
dg = pd.read_csv('LRM_48_hour_2.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_48 = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_48 = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_48 = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [33]:
dg = pd.read_csv('LRM_1_week_2.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_1w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_1w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_1w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [34]:
dg = pd.read_csv('LRM_2_week_2.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_2w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_2w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_2w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [35]:
dg = pd.read_csv('LRM_3_week_2.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_3w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_3w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_3w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [36]:
dg = pd.read_csv('LRM_4_week_2.csv')
dg = dg.replace([-np.inf,np.inf],np.nan)
X = dg[cols]
y = dg['dischargestatus']
X = X.fillna(X.mean())
X = np.nan_to_num(X)
an = training(X,y)
c_4w = mean_confidence_interval(an[0])
cm = confusion_matrix(an[1],an[2])
ppv_4w = round(float(cm[0][0])/(cm[0][0]+cm[0][1]),2)
npv_4w = round(float(cm[1][1])/(cm[1][0]+cm[1][1]),2)

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

In [39]:
l = [["1 Hour" ,c_a, ppv_1,npv_1],["6 Hour" ,c_6, ppv_6,npv_6],["12 Hour" ,c_12, ppv_12,npv_12],["48 Hour" ,c_48, ppv_48,npv_48],["1 Week" ,c_1w, ppv_1w,npv_1w],["2 Week" ,c_2w, ppv_2w,npv_3w],["3 Week" ,c_3w, ppv_3w,npv_3w],["4 Week" ,c_4w, ppv_4w,npv_4w]]

table = PrettyTable(['Parameter', 'AUC-ROC', 'PPV','NPV'])

for rec in l:
    table.add_row(rec)

print(table)

+-----------+-----------------------+------+------+
| Parameter |        AUC-ROC        | PPV  | NPV  |
+-----------+-----------------------+------+------+
|   1 Hour  | (0.797, 0.753, 0.841) | 0.86 | 0.5  |
|   6 Hour  | (0.795, 0.753, 0.836) | 0.82 | 0.78 |
|  12 Hour  | (0.835, 0.804, 0.867) | 0.83 | 0.88 |
|  48 Hour  | (0.873, 0.837, 0.908) | 1.0  | 0.86 |
|   1 Week  |  (0.844, 0.81, 0.878) | 1.0  | 0.29 |
|   2 Week  | (0.869, 0.835, 0.904) | 1.0  | 0.8  |
|   3 Week  | (0.889, 0.868, 0.911) | 0.93 | 0.8  |
|   4 Week  | (0.884, 0.853, 0.915) | 0.93 | 0.6  |
+-----------+-----------------------+------+------+
